Risk assessment with DSPy

Zero Shot

In [1]:
import dspy
from dspy.evaluate import Evaluate
from dspy import ChainOfThought
from dsp import Claude
from typing import List, Optional
import pandas as pd
import textwrap

/home/alberto/lab/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os
os.environ["DSP_CACHEBOOL"] = "False"
import phoenix as px
from phoenix.trace import using_project

phoenix_session = px.launch_app()
from openinference.instrumentation.dspy import DSPyInstrumentor
from openinference.instrumentation.anthropic import AnthropicInstrumentor

from phoenix.otel import register
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk.trace.export import ConsoleSpanExporter, SimpleSpanProcessor

endpoint = "http://127.0.0.1:6006/v1/traces"
tracer_provider = register(endpoint=endpoint, set_global_tracer_provider=False)
tracer_provider.add_span_processor(SimpleSpanProcessor(OTLPSpanExporter(endpoint)))

DSPyInstrumentor().instrument(tracer_provider=tracer_provider, skip_dep_check=True)
AnthropicInstrumentor().instrument(tracer_provider=tracer_provider)

🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix
🔭 OpenTelemetry Tracing Details 🔭
|  Phoenix Project: default
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: http://127.0.0.1:6006/v1/traces
|  Transport: HTTP
|  Transport Headers: {}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.



In [4]:
# Initialize the language model
#worker = dspy.OpenAI(model="gpt-3.5-turbo", model_type="chat", max_tokens=3000)
worker = Claude(model="claude-3-5-sonnet-20240620", max_tokens=3000)
#worker = dspy.Cohere(model="command-r-plus", max_tokens=3000)
#worker = dspy.HFModel(model = 'mistralai/Mistral-7B-Instruct-v0.2', max_tokens=3000)
dspy.configure(lm=worker)
dspy.settings.configure(backoff_time = 60)

In [5]:
# Input data
applicant_info = """
Name: John Doe
Age: 35
Annual Income: $75,000
Credit Score: 720
Existing Debts: $20,000 in student loans, $5,000 in credit card debt
Loan Amount Requested: $250,000 for a home mortgage
Employment: Software Engineer at Tech Corp for 5 years
"""

In [6]:
class ZeroShot(dspy.Module):
    """
    You are given a piece of text that contains information about an applicant. 
    Analyze the applicant's financial information and return a risk assessment.
    """
    def __init__(self):
        super().__init__()
        self.prog = dspy.Predict("question -> answer")

    def forward(self, applicant):
        return self.prog(question="Analyze the applicant's financial information and return a risk assessment. Applicant: " + applicant)

In [7]:
with using_project("zero_shot"):
    module = ZeroShot()
    response = module(applicant_info)
    print(f"ZeroShot response:\n {response}")

    prompt_used = worker.inspect_history(n=1)
    print(f"Prompt used: {prompt_used}")

 		You are using the client Claude, which will be removed in DSPy 2.6.
 		Changing the client is straightforward and will let you use new features (Adapters) that improve the consistency of LM outputs, especially when using chat LMs. 

 		Learn more about the changes and how to migrate at
 		https://github.com/stanfordnlp/dspy/blob/main/examples/migration.ipynb


ERROR:strawberry.execution:Unknown project: UHJvamVjdDoy

GraphQL request:4:3
3 | ) {
4 |   node(id: $id) {
  |   ^
5 |     __typename
Traceback (most recent call last):
  File "/home/alberto/lab/.venv/lib/python3.12/site-packages/graphql/execution/execute.py", line 530, in await_result
    return_type, field_nodes, info, path, await result
                                          ^^^^^^^^^^^^
  File "/home/alberto/lab/.venv/lib/python3.12/site-packages/strawberry/schema/schema_converter.py", line 750, in _async_resolver
    return await await_maybe(
           ^^^^^^^^^^^^^^^^^^
  File "/home/alberto/lab/.venv/lib/python3.12/site-packages/strawberry/utils/await_maybe.py", line 12, in await_maybe
    return await value
           ^^^^^^^^^^^
  File "/home/alberto/lab/.venv/lib/python3.12/site-packages/phoenix/server/api/queries.py", line 373, in node
    raise NotFound(f"Unknown project: {id}")
phoenix.server.api.exceptions.NotFound: Unknown project: UHJvamVjdDoy
ERROR:strawberry.exec

ZeroShot response:
 Prediction(
    answer="Based on the provided financial information for John Doe, here's a risk assessment:\n\nAnswer: Low to Moderate Risk\n\nReasons for this assessment:\n\n1. Positive factors:\n   - Stable employment: 5 years at Tech Corp as a Software Engineer\n   - Good credit score: 720 is considered good and indicates responsible credit management\n   - Solid annual income: $75,000 is a respectable salary for a 35-year-old\n   - Debt-to-income ratio: Existing debts are relatively low compared to annual income\n\n2. Potential concerns:\n   - Loan amount requested ($250,000) is more than 3 times the annual income, which is on the higher side\n   - Existing debts, while not excessive, do add to the overall financial obligations\n\n3. Recommendations:\n   - Verify the applicant's ability to make monthly mortgage payments comfortably\n   - Consider the local real estate market and ensure the loan-to-value ratio is appropriate\n   - Review the applicant's savings a

Labeled Few Shot

In [8]:
import json
from dspy.teleprompt import LabeledFewShot

In [9]:
class RiskAssessment(dspy.Signature):
    """Analyze the applicant's financial information and return a risk assessment."""
    question = dspy.InputField()
    applicant = dspy.InputField()
    answer = dspy.OutputField(desc="""
                              A thorough risk analysis about the applicant, justifying the assessment 
                              for each of the parameters considered from the applicant
                              """
                              )

In [10]:
class RiskAssessmentAgent(dspy.Module):
    def __init__(self):
        self.question = "Analyze the applicant's financial information and return a risk assessment."
        self.assess_risk = ChainOfThought(RiskAssessment, n=3)
    def forward(self, applicant:str):
        question = self.question
        applicant = applicant
        pred = self.assess_risk(question=question, applicant=applicant)

        return dspy.Prediction(answer = pred.answer)

In [11]:
# Load the training data
dataset = json.load(open("data/training_data.json", "r"))['examples']
trainset = [dspy.Example(question="Analyze the applicant's financial information and return a risk assessment", 
                         applicant=e['applicant'], 
                         answer=e['answer']) for e in dataset]

In [12]:
# Train
teleprompter = LabeledFewShot()
lfs_optimized_advisor = teleprompter.compile(RiskAssessmentAgent(), 
                                             trainset=trainset[3:]
                                            )

In [13]:
with using_project("labeled_few_shot"):
    response = lfs_optimized_advisor(applicant_info)
    wrapped_response = textwrap.fill(response.answer, width=70)
    print(f"LabeledFewShot Optimised response:\n {wrapped_response}")

LabeledFewShot Optimised response:
 Based on the analysis of John Doe's financial information, the risk
assessment is Low to Medium Risk. Here's the justification:  1.
Positive Factors:    - Good credit score (720) indicating responsible
credit management.    - Stable employment in a well-paying field.    -
Reasonable loan-to-income ratio for a mortgage.  2. Potential
Concerns:    - Existing debt, while not excessive, does add to his
overall financial obligations.    - The new debt-to-income ratio would
be on the higher side at 52.42%, which could strain his budget.  3.
Overall Assessment:    John Doe appears to be a responsible borrower
with a stable job and good credit history. The mortgage amount
requested is in line with his income, although it will significantly
increase his debt load. The main risk factor is the potentially high
debt-to-income ratio after taking on the mortgage.  Recommendation:
The lender should feel relatively confident in John's ability to repay
the loan, but 

In [14]:
prompt_used = worker.inspect_history(n=1)
print(f"Prompt used: {prompt_used}")




Analyze the applicant's financial information and return a risk assessment.

---

Follow the following format.

Question: ${question}

Applicant: ${applicant}

Reasoning: Let's think step by step in order to ${produce the answer}. We ...

Answer: A thorough risk analysis about the applicant, justifying the assessment for each of the parameters considered from the applicant

---

Question: Analyze the applicant's financial information and return a risk assessment
Applicant: Name: Richard Turner Age: 45 Annual Income: $95,000 Credit Score: 620 Existing Debts: $20,000 in credit card debt Loan Amount Requested: $30,000 for home renovations Employment: Engineer at a manufacturing company for 15 years
Answer: Based on the analysis of Richard Turner's financial information, the risk assessment is Medium. Here's the justification for this assessment: 1. Income: Richard's annual income of $95,000 is relatively high, providing a good foundation for loan repayment. 2. Employment Stability: 15 

# Bootstrap Few Shot

In [25]:
from dspy.teleprompt import BootstrapFewShot

In [26]:
# Define the signature for automatic assessments.
class Assess(dspy.Signature):
    """Assess the quality of a risk assessment along the specified dimension."""

    assessed_text = dspy.InputField()
    assessment_question = dspy.InputField()
    assessment_answer = dspy.OutputField(desc="Yes or No")

In [27]:
def risk_assessment_metric(gold, pred, trace=None):
    applicant, risk_assessment = gold.applicant, pred.answer

    correct = f"The text above should provide a risk assessment for `{applicant}`. Does it do so? Answer with Yes or No."
    complete = f"Does the text above make for a reasoned assessment across all areas mentioned in `{applicant}`? Answer with Yes or No."

    with dspy.context(lm=worker):
        correct =  dspy.Predict(Assess)(assessed_text=risk_assessment, assessment_question=correct)
        complete = dspy.Predict(Assess)(assessed_text=risk_assessment, assessment_question=complete)

    correct, complete = [m.assessment_answer.split()[0].lower() == 'yes' for m in [correct, complete]]
    score = (correct + complete) if correct else 0

    if trace is not None: return score >= 2
    return score / 2.0

In [28]:
# Load the training data
dataset = json.load(open("data/training_data.json", "r"))['examples']
trainset = [dspy.Example(question="Analyze the applicant's financial information and return a risk assessment", 
                         applicant=e['applicant'], 
                         answer=e['answer']) for e in dataset]

In [29]:
with using_project("bootstrap_few_shot"):
    bfs_trainset = [x.with_inputs('applicant') for x in trainset]
    config = dict(max_bootstrapped_demos=4, max_labeled_demos=4)
    bfs_optimized = BootstrapFewShot(metric=risk_assessment_metric, **config)
    bfs_optimized_advisor = bfs_optimized.compile(RiskAssessmentAgent(),
                                                trainset=bfs_trainset)

ERROR:dspy.teleprompt.bootstrap:2024-10-07T12:51:12.708074Z [error    ] Failed to run or to evaluate example Example({'question': "Analyze the applicant's financial information and return a risk assessment", 'applicant': 'Name: Sarah Johnson\nAge: 45\nAnnual Income: $120,000\nCredit Score: 620\nExisting Debts: $10,000 in credit card debt\nLoan Amount Requested: $10,000 for a personal loan\nEmployment: Self-employed as a freelance writer for 10 years', 'answer': "Based on the analysis, Sarah Johnson presents a Medium to High Risk for the following reasons: 1. Income Stability: While her annual income of $120,000 is substantial, her self-employment status as a freelance writer introduces some uncertainty about income stability. 2. Credit Score: Her credit score of 620 is concerning, as it falls in the fair to poor range. This suggests a history of credit management issues and increases the risk of default. 3. Existing Debt: The $10,000 in credit card debt, combined with the fair-to-poor 

Bootstrapped 4 full traces after 10 examples in round 0.


In [30]:
with using_project("bootstrap_few_shot"):
    response = bfs_optimized_advisor(applicant_info)
    wrapped_response = textwrap.fill(response.answer, width=70)
    print(f"BootstrapFewShot Optimised response:\n {wrapped_response}")

BootstrapFewShot Optimised response:
 Based on the analysis of John Doe's financial information, the risk
assessment is Medium Risk. Here's the justification for this
assessment:  1. Income and Employment: John's solid income of $75,000
and stable employment as a Software Engineer for 5 years are positive
factors, indicating a reliable source for loan repayment.  2. Credit
Score: His credit score of 720 is good, suggesting responsible credit
management and reducing the risk of default.  3. Existing Debt: The
$25,000 in existing debts (student loans and credit card) is notable
but not excessive given his income. However, it does impact his
overall debt burden.  4. Debt-to-Income Ratio: The current DTI of
33.3% is at the higher end of what's typically considered acceptable.
The projected DTI of 52.4% after taking on the mortgage is a
significant concern, as it's well above the 43% maximum that many
lenders prefer for qualified mortgages.  5. Loan Amount and Purpose:
While a $250,000 mort

In [31]:
prompt_used = worker.inspect_history(n=1)
print(f"Prompt used: {prompt_used}")




Analyze the applicant's financial information and return a risk assessment.

---

Follow the following format.

Question: ${question}

Applicant: ${applicant}

Reasoning: Let's think step by step in order to ${produce the answer}. We ...

Answer: A thorough risk analysis about the applicant, justifying the assessment for each of the parameters considered from the applicant

---

Question: Analyze the applicant's financial information and return a risk assessment.

Applicant: Name: John Smith Age: 35 Annual Income: $80,000 Credit Score: 750 Existing Debts: $0 Loan Amount Requested: $50,000 for a car loan Employment: Software Engineer at XYZ Corp for 5 years

Reasoning: Let's think step by step in order to Reasoning: Let's think step by step in order to produce a comprehensive risk assessment for John Smith: 1. Age: At 35, John is in his prime working years, which is a positive factor. 2. Annual Income: $80,000 is a solid income, indicating good earning potential and ability to repay.

In [32]:
bfs_optimized_advisor.save('bfs_optimized_advisor_compiled.json')

[('assess_risk', Predict(StringSignature(question, applicant -> rationale, answer
    instructions="Analyze the applicant's financial information and return a risk assessment."
    question = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Question:', 'desc': '${question}'})
    applicant = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Applicant:', 'desc': '${applicant}'})
    rationale = Field(annotation=str required=True json_schema_extra={'prefix': "Reasoning: Let's think step by step in order to", 'desc': '${produce the answer}. We ...', '__dspy_field_type': 'output'})
    answer = Field(annotation=str required=True json_schema_extra={'desc': '\n                              A thorough risk analysis about the applicant, justifying the assessment \n                              for each of the parameters considered from the applicant\n                              ', '__dspy_field_type': 

Bootstrap Few Shot with Advanced Metrics using G-Eval (https://arxiv.org/abs/2303.16634)

In [35]:
from deepeval.metrics import GEval, BiasMetric, AnswerRelevancyMetric
from deepeval.test_case import LLMTestCaseParams

In [43]:
def risk_assessment_metric_adv(gold, pred, trace=None):
    applicant, risk_assessment = gold.applicant, pred.answer
    print(f"applicant: {applicant}")
    print(f"risk_assessment: {risk_assessment}")
    test_case = LLMTestCaseParams(
            input=applicant,
            actual_output=risk_assessment
    )
    with dspy.context(lm=worker):
        bias_metric = BiasMetric(
            model=worker,
            threshold=0.7,
            include_reason=True,
            async_mode=False
        )
        bias_metric.measure(test_case)
        answer_relevancy_metric = AnswerRelevancyMetric(
            model=worker,
            threshold=0.7,
            include_reason=True,
            async_mode=False
        )
        answer_relevancy_metric.measure(test_case)
        coherence_metric = GEval(
            model=worker,
            name="coherence",
            criteria="Coherence - determine if the actual output is coherent with the input",
            evaluation_params=[LLMTestCaseParams.INPUT, LLMTestCaseParams.ACTUAL_OUTPUT],
            evaluation_steps=[
                "Check if the actual output is logically consistent with the input on the basis of the instructions from the input",
                "Assess if the actual output is relevant to the input and maintains a logical flow on the basis of the instructions from the input",
            ],
            async_mode=False
        )
        coherence_metric.measure(test_case)
    
    
    score = bias_metric.score + answer_relevancy_metric.score + coherence_metric.score
    
    if trace is not None: return score >= 3
    return score / 3.0

In [44]:
# Load the training data
dataset = json.load(open("data/training_data.json", "r"))['examples']
trainset = [dspy.Example(question="Analyze the applicant's financial information and return a risk assessment", 
                         applicant=e['applicant'], 
                         answer=e['answer']) for e in dataset]

In [45]:
with using_project("bootstrap_few_shot_adv_metrics"):
    bfs_trainset = [x.with_inputs('applicant') for x in trainset]
    config = dict(max_bootstrapped_demos=4, max_labeled_demos=4)
    bfs_optimized_adv_metrics = BootstrapFewShot(metric=risk_assessment_metric_adv, **config)
    bfs_optimized_advisor_adv_metrics = bfs_optimized_adv_metrics.compile(RiskAssessmentAgent(),
                                                trainset=bfs_trainset)

 15%|█▍        | 4/27 [04:42<27:06, 70.70s/it]


InternalServerError: Error code: 529 - {'type': 'error', 'error': {'type': 'overloaded_error', 'message': 'Overloaded'}}

MIPRO

In [23]:
from dspy.teleprompt import MIPROv2

In [24]:
with using_project("mipro_v2"):
    mipro_trainset = [x.with_inputs('applicant') for x in trainset]
    config = dict(prompt_model=worker, task_model=worker, num_candidates=2, init_temperature=0.1)
    eval_kwargs = dict(num_threads=1, display_progress=True, display_table=0)
    mipro_optimized = MIPROv2(metric=risk_assessment_metric, **config)
    mipro_optimized_advisor = mipro_optimized.compile(RiskAssessmentAgent(),
                                                trainset=mipro_trainset,
                                                max_bootstrapped_demos=2,
                                                max_labeled_demos=2,
                                                num_trials=2,
                                                minibatch=False
                                                )

Projected Language Model (LM) Calls

Please be advised that based on the parameters you have set, the maximum number of LM calls is projected as follows:


- Prompt Model: 10 data summarizer calls + 2 * 1 lm calls in program + (2) lm calls in program aware proposer = 14 prompt model calls
- Task Model: 21 examples in val set * 2 batches * # of LM calls in your program = (12 * # of LM calls in your program) task model calls

Estimated Cost Calculation:

Total Cost = (Number of calls to task model * (Avg Input Token Length per Call * Task Model Price per Input Token + Avg Output Token Length per Call * Task Model Price per Output Token) 
            + (Number of calls to prompt model * (Avg Input Token Length per Call * Task Prompt Price per Input Token + Avg Output Token Length per Call * Prompt Model Price per Output Token).

For a preliminary estimate of potential costs, we recommend you perform your own calculations based on the task
and prompt models you intend to use. If the projec

Average Metric: 15.0 / 21  (71.4): 100%|██████████| 21/21 [02:48<00:00,  8.03s/it]
/home/alberto/lab/.venv/lib/python3.12/site-packages/optuna/_experimental.py:30: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(


Default program score: 71.43

==> STEP 3: FINDING OPTIMAL PROMPT PARAMETERS <==
In this step, we will evaluate the program over a series of trials with different combinations of instructions and few-shot examples to find the optimal combination. Bayesian Optimization will be used for this search process.

===== Trial 1 / 2 =====


Average Metric: 2.0 / 4  (50.0):  14%|█▍        | 3/21 [00:46<03:17, 10.98s/it]ERROR:backoff:Giving up request(...) after 1 tries (anthropic.RateLimitError: Error code: 429 - {'type': 'error', 'error': {'type': 'rate_limit_error', 'message': 'Number of request tokens has exceeded your daily rate limit (https://docs.anthropic.com/en/api/rate-limits); see the response headers for current usage. Please reduce the prompt length or the maximum tokens requested, or try again later. You may also contact sales at https://www.anthropic.com/contact-sales to discuss your options for a rate limit increase.'}})
ERROR:dspy.evaluate.evaluate:2024-10-05T19:22:57.729794Z [error    ] Error for example in dev set: 		 Error code: 429 - {'type': 'error', 'error': {'type': 'rate_limit_error', 'message': 'Number of request tokens has exceeded your daily rate limit (https://docs.anthropic.com/en/api/rate-limits); see the response headers for current usage. Please reduce the prompt length or the maximum tokens

RateLimitError: Error code: 429 - {'type': 'error', 'error': {'type': 'rate_limit_error', 'message': 'Number of request tokens has exceeded your daily rate limit (https://docs.anthropic.com/en/api/rate-limits); see the response headers for current usage. Please reduce the prompt length or the maximum tokens requested, or try again later. You may also contact sales at https://www.anthropic.com/contact-sales to discuss your options for a rate limit increase.'}}

In [46]:
response = mipro_optimized_advisor(applicant_info)
wrapped_response = textwrap.fill(response.answer, width=70)
print(f"MIPRO Optimised response:\n {wrapped_response}")

MIPRO Optimised response:
 Based on the analysis of John Doe's financial information, here's a
risk assessment:  1. Income Stability: Low Risk    John's steady
employment as a Software Engineer for 5 years at Tech Corp suggests a
stable income source. The tech industry generally offers good job
security and potential for career growth.  2. Credit History: Low Risk
With a credit score of 720, John demonstrates responsible credit
management. This score is likely to qualify him for favorable interest
rates and loan terms.  3. Debt Management: Low to Moderate Risk
John's existing debts are manageable relative to his income. The debt-
to-income ratio of 33.3% is within acceptable limits, though it's
approaching the higher end of what many lenders prefer.  4. Loan
Affordability: Low to Moderate Risk    The requested loan amount of
$250,000 results in a loan-to-income ratio of 3.33:1, which is
generally considered acceptable for a mortgage. However, it's on the
higher side and may stretch Joh

In [50]:
response = mipro_optimized_advisor(applicant_info)
print(f"MIPRO Optimised response:\n {response}")

MIPRO Optimised response:
 Prediction(
    answer="Based on the analysis of John Doe's financial information, here's a risk assessment:\n\n1. Income Stability: Low Risk\n   John's steady employment as a Software Engineer for 5 years at Tech Corp suggests a stable income source. The tech industry generally offers good job security and potential for career growth.\n\n2. Credit History: Low Risk\n   With a credit score of 720, John demonstrates responsible credit management. This score is likely to qualify him for favorable interest rates and loan terms.\n\n3. Debt Management: Low to Moderate Risk\n   John's existing debts are manageable relative to his income. The debt-to-income ratio of 33.3% is within acceptable limits, though it's approaching the higher end of what many lenders prefer.\n\n4. Loan Affordability: Low to Moderate Risk\n   The requested loan amount of $250,000 results in a loan-to-income ratio of 3.33:1, which is generally considered acceptable for a mortgage. However, it

In [52]:
prompt_used = worker.inspect_history(n=1)
print(f"Prompt used: {prompt_used}")




Analyze the applicant's financial information and return a risk assessment.

---

Follow the following format.

Question: ${question}

Applicant: ${applicant}

Reasoning: Let's think step by step in order to ${produce the answer}. We ...

Answer: A thorough risk analysis about the applicant, justifying the assessment for each of the parameters considered from the applicant

---

Question: Analyze the applicant's financial information and return a risk assessment.

Applicant: Name: John Doe Age: 35 Annual Income: $75,000 Credit Score: 720 Existing Debts: $20,000 in student loans, $5,000 in credit card debt Loan Amount Requested: $250,000 for a home mortgage Employment: Software Engineer at Tech Corp for 5 years

Reasoning: Let's think step by step in order toReasoning: Let's think step by step in order to produce a comprehensive risk assessment for John Doe's loan application:

1. Age:
   - At 35, John is in his prime working years, which is generally favorable.
   - He likely has a 

In [48]:
mipro_optimized_advisor.save('mipro_optimized_advisor_compiled.json')

[('assess_risk', Predict(StringSignature(question, applicant -> rationale, answer
    instructions="Analyze the applicant's financial information and return a risk assessment."
    question = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Question:', 'desc': '${question}'})
    applicant = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Applicant:', 'desc': '${applicant}'})
    rationale = Field(annotation=str required=True json_schema_extra={'prefix': "Reasoning: Let's think step by step in order to", 'desc': '${produce the answer}. We ...', '__dspy_field_type': 'output'})
    answer = Field(annotation=str required=True json_schema_extra={'desc': '\n                              A thorough risk analysis about the applicant, justifying the assessment \n                              for each of the parameters considered from the applicant\n                              ', '__dspy_field_type': 